# Train prediction models for grating orientation 
* convolutional neural network, input features_SA and features RA of size (n_pins, n_pins) representing firing rate on an n_pins x n_pins square array                
* training data concatenated over grating stimuli of periods 0-5mm (stimuli 1 to 8) and physical orientation angle of the stimulus (90, 180, 270, 36)
* only the yaw relative to the grating orientation is kept as a label; range (-90,90) rescaled to (0,1)
* i.e. prediction independent of grating period and physical orientation of the stimulus relative to the robot

To run, first edit dir_data to path where data is stored; experiment_2/process in this directory should be run first.

In [36]:
import pickle, os
import numpy as np

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [37]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-gratings-jrsi2022/experiment_2"
n_stimuli = 8
n_angles = 4
stimuli = [f'{i}' for i in range(n_stimuli)] # do not include smooth (0) stimulus
angles = [f'{90*a}' for a in range(n_angles)]
n_pins = 19

In [38]:
X_train_SA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/X_train_sa") for j in range(n_angles)] for i in range(n_stimuli)]
y_train_SA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/y_train_sa") for j in range(n_angles)] for i in range(n_stimuli)]
X_val_SA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/X_val_sa") for j in range(n_angles)] for i in range(n_stimuli)]
y_val_SA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/y_val_sa") for j in range(n_angles)] for i in range(n_stimuli)]

X_train_RA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/X_train_ra") for j in range(n_angles)] for i in range(n_stimuli)]
y_train_RA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/y_train_ra") for j in range(n_angles)] for i in range(n_stimuli)]
X_val_RA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/X_val_ra") for j in range(n_angles)] for i in range(n_stimuli)]
y_val_RA = [[open_obj(dir_data + rf"/processed/train_data/{stimuli[i]}/{angles[j]}/y_val_ra") for j in range(n_angles)] for i in range(n_stimuli)]

In [39]:
X_train_SA = np.reshape(np.stack(X_train_SA), (-1,n_pins,n_pins))
y_train_SA = (np.reshape(np.stack(y_train_SA), (-1)) + 90)/180
X_val_SA = np.reshape(np.stack(X_val_SA), (-1,n_pins,n_pins))
y_val_SA = (np.reshape(np.stack(y_val_SA), (-1)) + 90)/180

X_train_RA = np.reshape(np.stack(X_train_RA), (-1,n_pins,n_pins))
y_train_RA = (np.reshape(np.stack(y_train_RA), (-1)) + 90)/180
X_val_RA = np.reshape(np.stack(X_val_RA), (-1,n_pins,n_pins))
y_val_RA = (np.reshape(np.stack(y_val_RA), (-1)) + 90)/180

In [40]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session
from keras import optimizers, regularizers, callbacks
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from numpy.random import seed
from tensorflow import set_random_seed

### Define model

In [41]:
def define_model(X_train, y_train, X_val, y_val, es, cp):
    
    clear_session()
        
    config = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8))
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    set_session(session)
    
    seed(1)
    set_random_seed(2)

    model = Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', padding="same", input_shape=(n_pins,n_pins,1)))
    model.add(MaxPooling2D((2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding="valid"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(32, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(16, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer = optimizers.rmsprop(lr=1e-4), metrics=['mae'])
    model.fit(np.expand_dims(X_train, axis=3), y_train, validation_data=(np.expand_dims(X_val, axis=3), y_val), epochs=100, batch_size=32, shuffle=True, callbacks=[cp,es])

In [42]:
os.makedirs(dir_data + r"/models/orientation_SA/", exist_ok=True)  
cp = callbacks.ModelCheckpoint(dir_data + r"/models/orientation_SA/model_{epoch:02d}_{val_mae:.2f}.hdf5", monitor='val_mae', save_best_only=True)
es = callbacks.EarlyStopping(monitor='val_mae', patience=20, restore_best_weights=True)
define_model(X_train_SA, y_train_SA, X_val_SA, y_val_SA, es, cp)

os.makedirs(dir_data + r"/models/orientation_RA/", exist_ok=True)  
cp = callbacks.ModelCheckpoint(dir_data + r"/models/orientation_RA/model_{epoch:02d}_{val_mae:.2f}.hdf5", monitor='val_mae', save_best_only=True)
es = callbacks.EarlyStopping(monitor='val_mae', patience=20, restore_best_weights=True)
define_model(X_train_RA, y_train_RA, X_val_RA, y_val_RA, es, cp)

Train on 5984 samples, validate on 2016 samples
Epoch 1/500
5984/5984 [==============================] - 2s 407us/step - loss: 1.4325 - mae: 0.7793 - val_loss: 0.8689 - val_mae: 0.6016
Epoch 2/500
5984/5984 [==============================] - 2s 274us/step - loss: 1.0917 - mae: 0.6457 - val_loss: 0.5963 - val_mae: 0.3570
Epoch 3/500
5984/5984 [==============================] - 2s 273us/step - loss: 0.9571 - mae: 0.5810 - val_loss: 0.5248 - val_mae: 0.2981
Epoch 4/500
5984/5984 [==============================] - 2s 278us/step - loss: 0.8305 - mae: 0.5224 - val_loss: 0.5210 - val_mae: 0.3105
Epoch 5/500
5984/5984 [==============================] - 2s 273us/step - loss: 0.7777 - mae: 0.4947 - val_loss: 0.4972 - val_mae: 0.3016
Epoch 6/500
5984/5984 [==============================] - 2s 278us/step - loss: 0.7094 - mae: 0.4610 - val_loss: 0.4519 - val_mae: 0.2768
Epoch 7/500
5984/5984 [==============================] - 2s 297us/step - loss: 0.6612 - mae: 0.4407 - val_loss: 0.4131 - val_mae: 